In [16]:
from google.colab import drive

import csv
import pandas as pd
import ast
import re

In [17]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
base_path = '/content/drive/MyDrive/23-1_online/빅데이터 처리/팀플/Big Data Processing'

In [25]:
kaggle_path = base_path + '/data/kaggle_data.csv'
steam_1_path = base_path + '/data/steam_data(to 40000).csv'
steam_2_path = base_path + '/data/steam_data(40000 to 58852).csv'
vg_1_path = base_path + '/data/vg_data(to 30000).csv'
vg_2_path = base_path + '/data/vg_data(30000 to 58852).csv'

combined_path = base_path + '/data/combined_output.csv'

In [28]:
with open(kaggle_path, 'r', encoding='cp949', errors='replace') as kaggle_file, \
open(steam_1_path, 'r', encoding='utf8', errors='replace') as steam_1_file, \
open(steam_2_path, 'r', encoding='utf8', errors='replace') as steam_2_file, \
open(vg_1_path, 'r', encoding='cp949', errors='replace') as vg_1_file, \
open(vg_2_path, 'r', encoding='cp949', errors='replace') as vg_2_file, \
open(combined_path, 'r', encoding='cp949', errors='replace') as combined_file:
#     kaggle_df = pd.read_csv(kaggle_file)
    steam_1_df = pd.read_csv(steam_1_file)
    steam_2_df = pd.read_csv(steam_2_file)
#     vg_1_df = pd.read_csv(vg_1_file)
#     vg_2_df = pd.read_csv(vg_2_file)
    combined_df = pd.read_csv(combined_file)

<ipython-input-28-bbab749aa557>:12: DtypeWarning: Columns (40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv(combined_file)


In [29]:
steam_1_df = steam_1_df.iloc[:10, 5:]
steam_2_df = steam_2_df.iloc[:, 5:]
# vg_1_df = vg_1_df.iloc[:, 3:]
# vg_2_df = vg_2_df.iloc[1:, 3:]

In [31]:
print(steam_1_df)

                                   Currently popular  \
0  ['Not Recommended, \n\t\t\t\t\tAt first glance...   
1  ["Recommended, \n\t\t\t\t\tOne Finger Death Pu...   
2                                                 []   
3  ['Recommended, \nIntroductionWhile 2D platform...   
4  ['Not Recommended, \n\t\t\t\t\tThere’s a reaso...   
5  ['Not Recommended, \n\t\t\t\t\tPlayed this gam...   
6  ["Not Recommended, \n\t\t\t\t\tSo, this was on...   
7                                                 []   
8  ["Recommended, \n\t\t\t\t\tI came across this ...   
9  ["Recommended, \n\t\t\t\t\tDespite the name Ro...   

                                    Game Description  
0  Galactic Bowling is an exaggerated and stylize...  
1  THE LAW!!Looks to be a showdown atop a train. ...  
2  Jolt Project:The army now has a new robotics p...  
3  In bizarre lands, play as a viscous glob of wa...  
4  ABOUT THE GAMEPlay as a hacker who has arrange...  
5  Feel tired of auto-fight? Feel tired of boring... 

In [32]:
def parse_and_join_list(s, delimiter=', '):
    if isinstance(s, str) and s.startswith('[') and s.endswith(']'):
        try:
            lst = ast.literal_eval(s)
            if isinstance(lst, list):
                cleaned_lst = [elem.replace('Not Recommended, ', '').replace('Recommended, ', '') for elem in lst]
                return delimiter.join(cleaned_lst)
        except (ValueError, SyntaxError):
            pass
    return s


steam_1_df['Currently popular'] = steam_1_df['Currently popular'].apply(parse_and_join_list)


In [35]:
steam_1_df['Currently popular'] = steam_1_df['Currently popular'].apply(parse_and_join_list)
print(steam_1_df)

                                   Currently popular  \
0  \n\t\t\t\t\tAt first glance, Galactic Bowling ...   
1  \n\t\t\t\t\tOne Finger Death PunchBut it's ado...   
2                                                      
3  \nIntroductionWhile 2D platformer and puzzle g...   
4  \n\t\t\t\t\tThere’s a reason why visual novels...   
5  \n\t\t\t\t\tPlayed this game about 10  years a...   
6  \n\t\t\t\t\tSo, this was one of the Steam Next...   
7                                                      
8  \n\t\t\t\t\tI came across this game on Play St...   
9  \n\t\t\t\t\tDespite the name Rotten Edition, t...   

                                    Game Description  
0  Galactic Bowling is an exaggerated and stylize...  
1  THE LAW!!Looks to be a showdown atop a train. ...  
2  Jolt Project:The army now has a new robotics p...  
3  In bizarre lands, play as a viscous glob of wa...  
4  ABOUT THE GAMEPlay as a hacker who has arrange...  
5  Feel tired of auto-fight? Feel tired of boring... 

In [36]:
def remove_escape_sequences(s):
    if isinstance(s, str):
        return s.replace('\n', ' ').replace('\t', ' ')
    else:
        return s

In [38]:
string_columns = steam_1_df.select_dtypes(include='object').columns
steam_1_df[string_columns] = steam_1_df[string_columns].applymap(remove_escape_sequences)

print(steam_1_df)

                                   Currently popular  \
0        At first glance, Galactic Bowling appear...   
1        One Finger Death PunchBut it's adorable!...   
2                                                      
3   IntroductionWhile 2D platformer and puzzle ga...   
4        There’s a reason why visual novels have ...   
5        Played this game about 10  years ago for...   
6        So, this was one of the Steam Next Fest ...   
7                                                      
8        I came across this game on Play Store. I...   
9        Despite the name Rotten Edition, this ga...   

                                    Game Description  
0  Galactic Bowling is an exaggerated and stylize...  
1  THE LAW!!Looks to be a showdown atop a train. ...  
2  Jolt Project:The army now has a new robotics p...  
3  In bizarre lands, play as a viscous glob of wa...  
4  ABOUT THE GAMEPlay as a hacker who has arrange...  
5  Feel tired of auto-fight? Feel tired of boring... 

In [41]:
sample_text_data_path = base_path + '/data/sample_text_data.csv'

In [42]:
steam_1_df.to_csv(sample_text_data_path, index = False, encoding = 'UTF8')